In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import loadmat
import numpy.linalg as la
import os

##############################################################################

# Function that esnsures a quaternion array is continuous in sign 
def makeContinuous(Q_array):
    
    N = Q_array.shape[1]

    for i in range(1, N):
        if la.norm( Q_array[:,i] - Q_array[:, i-1]) > la.norm( - Q_array[:, i] - Q_array[:, i-1] ):
            Q_array[:, i] = - Q_array[:, i]

    return Q_array

##############################################################################

# Defining the Gaussian time interpolation function
def gaussian_time_interpolation(t, times, values, sigma):

    # known time-value pairs
    times = np.asarray(times).flatten()   
    values = np.asarray(values).flatten()    

    t = np.atleast_1d(t).flatten()  #new time points 

    # We use the Nadaraya–Watson kernel regression (or “kernel smoothing”) in a time interpolation context.
    kernel_weights = np.exp(-((t[:, None] - times[None, :])**2) / (2 * sigma**2))       # Using Gaussian kernel 
    # Apply the Gaussian kernel function to compute weights between each t[i] and each times[j].

    # Checking for mismatched shapes
    if kernel_weights.shape[1] != values.shape[0]:  
        raise ValueError(f"Shape mismatch: weights {kernel_weights.shape}, values {values.shape}")


    kernel_weighted_sum = np.sum(kernel_weights * values, axis=1)
    kernel_sum_weights = np.sum(kernel_weights, axis=1)   # Computing here the Kernel-Weighted Sums

    interpolated = kernel_weighted_sum / kernel_sum_weights    # Computing the interpolated values

    return interpolated if interpolated.size > 1 else interpolated[0]


##############################################################################

#### COMPUTING VELOCITY REGARDING THE QUATERNION DATA ####   

## --- ω = 2 * J(q) * q_dot ---

# Computes the mapping matrix J(q) for a quaternion q = [q0, q1, q2, q3].
def compute_J(q):
   
    q0, q1, q2, q3 = q

    # Jacobian matrix
    J = np.array([
        [-q1, q0,  -q3, q2],
        [-q2, q3,  q0,  -q1],
        [-q3, -q2, q1,  q0]
    ])

    return J

# Computes the angular velocity ω for each quaternion in Q and its time derivative dQ
# using the formula: ω = 2 * J(q) * q_dot
def compute_angular_velocity(Q, dQ):
    """
    Parameters:
      Q : numpy.ndarray of shape (N, 4)
          Array of quaternions [q0, q1, q2, q3].
      dQ : numpy.ndarray of shape (N, 4)
          Array of quaternion time derivatives.
    """

    N = Q.shape[0]  # Number of quaternions
    omega = np.zeros((N, 3))  # Initializing the angular velocity array
    
    for i in range(N):

        q = Q[i]
        q_dot = dQ[i]

        J = compute_J(q)

        omega[i] = 2.0 * J @ q_dot
        
    return omega
#  Returns ω (Angular velocity vectors) and have the shape -> (N, 3)

##############################################################################

# Computes the supremum velocity of a given velocity data.
def supremum_v(velocity_data):

    # Sum of absolute values for each time-step
    sum_abs_each_t = np.sum(np.abs(velocity_data), axis=1)
    sum_abs_each_t = np.max(np.abs(velocity_data), axis=1)

    # Return the maximum across all time steps
    return np.max(np.abs(velocity_data))

##############################################################################


file_path = "./new_primitives/swing_tilt_twist_slide_demo.mat"
data = loadmat(file_path)

# Time array
t = data['t'].flatten()

sampling_rate = 20  # 20 samples per second

motion_duration = np.max(t) - np.min(t)  # Total duration of motion
print(np.max(t))
num_interp_points = int(motion_duration * sampling_rate)  # Dynamically calculate number of points

t_interp = np.linspace(np.min(t), np.max(t), num_interp_points)


sigma = 0.1

# The original data 'x': Nx7 from which -> x_1..x_3 = position, x_4..x_7 = quaternion
# --- Extract Position (P) and Orientation (Q) ---
P = data['x'][:, 0:3]       
Q = data['x'][:, 3:7]  
Qf = makeContinuous(Q.T).T  # shape (N, 4)


# --------------------------------------------------------------------------------------------

#### COMPUTING VELOCITY REGARDING THE POSITION DATA ####   

# --- Numerical Differentiation and Gaussian Approximation ---

p_dot = np.gradient(P, t, axis=0)  # Computing the velocity (time derivative) for the position ---> p_dot
# using the np.gradient cause this method automatically handles central differences for interior points and forward/backward differences for the endpoints.


Q = data['x'][:, 3:7]  
Qf = makeContinuous(Q.T).T   # Making the quaternion data continuous

dQ = np.gradient(Qf, t, axis=0)  # Computing the time derivative of the quaternion data

# Compute the angular velocity ω using the quaternions Q and their time derivatives dQ
w = compute_angular_velocity(Qf, dQ)

# Initialize an array to store the interpolated angular velocity values (w_hat)
w_hat = np.zeros((num_interp_points, 3))  # Shape (num_interp_points, 3)

# Defining the libraries to store the results. 
interpolated_results_regarding_position = {}
interpolated_results_regarding_quaternion = {}

# Apply the Gaussian interpolation 
# range(6) since we have 3 values for position and 3 values for angular velocity
for i in range(6):

    if i >= 3:

        angular_vel_values = gaussian_time_interpolation(t_interp, t, w[:, i-3], sigma)
        interpolated_results_regarding_quaternion[f"w_{i-2}"] = angular_vel_values

    else:

        position_values = gaussian_time_interpolation(t_interp, t, p_dot[:, i], sigma)
        interpolated_results_regarding_position[f"p_{i+1}"] = position_values


# --- Supremum Velocity Calculation ---

# Compute supremum for linear velocity (p_dot)
a_linear = supremum_v(p_dot)

# Compute supremum for angular velocity (w)
a_angular = supremum_v(w)

a_global = max(a_linear, a_angular) # Global supremum velocity for both linear and angular velocities. 
# a_global finds the largest absolute velocity seen in either p_dot or w.

# Plotting position derivative channels (linear velocity)
for i in range(3):
    plt.figure(figsize=(12, 5))
    plt.plot(t, p_dot[:, i], 'o', label=f'Original Data p_{i+1}', alpha=0.5)
    plt.plot(t_interp, interpolated_results_regarding_position[f"p_{i+1}"],
             label=f'Interpolated p_{i+1}', linestyle='--')
    plt.xlabel("Time")
    plt.ylabel(f"Value (p_{i+1})")
    plt.title(f"Gaussian Time Interpolation for p_{i+1} (sigma={sigma})")
    plt.legend()
    plt.grid(True)

    plt.ylim([-a_global, a_global])  # linear scaling 

    plt.show()


# Plotting angular velocity
for j in range(3):
    plt.figure(figsize=(12, 5))
    plt.plot(t, w[:, j], 'o', label=f'Original Data w_{j+1}', alpha=0.5)
    plt.plot(t_interp, interpolated_results_regarding_quaternion[f"w_{j+1}"],
             label=f'Interpolated w_{j+1}', linestyle='--')
    plt.xlabel("Time")
    plt.ylabel(f"Value (w_{j+1})")
    plt.title(f"Gaussian Time Interpolation for w_{j+1} (sigma={sigma})")
    plt.legend()
    plt.grid(True)

    plt.ylim([-a_global, a_global])  # angular scaling

    plt.show()


# Create DataFrames from the interpolated results dictionaries.
df_position = pd.DataFrame(interpolated_results_regarding_position)
df_quaternion = pd.DataFrame(interpolated_results_regarding_quaternion)


# Create DataFrames from the interpolated results dictionaries.
df_position = pd.DataFrame(interpolated_results_regarding_position)
df_quaternion = pd.DataFrame(interpolated_results_regarding_quaternion)

# Concatenate the two DataFrames 
df_results = pd.concat([df_position, df_quaternion], axis=1)

# save_directory = "./corl_primitives" 
# csv_filename = os.path.join(save_directory, "complex_motion_20.csv")

# df_results.to_csv(csv_filename, index=False)

# print(f"Results saved to {csv_filename}")


14.482000000001506
